In [38]:
from itertools import chain
# To make it work on OS X
import matplotlib
matplotlib.use('TkAgg')
# --
import ujson as json
import multiprocessing as mp
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_fscore_support

from helpers import disaggregated_df
from helpers import aggregated_df
from helpers import dummify_df

%matplotlib inline
sns.set(font='monospace')

/Users/svebor/anaconda/envs/memex_qpr_april_2016_cp1_python3/lib/python3.5/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [59]:
def format_jsns_to_key_count(jsns):
    records = []
    for record in jsns:
        key = list(record.keys())
        #print(key[0],len(record[key[0]]))
        records.append((key[0],np.asarray([len(record[key[0]])])))
    return records

In [103]:
jsns = [json.loads(x) for x in open('../../data/orig_ht_data/ht_training_UPDATED', 'r')]

{'ethnicity': {'attribs': {'type': 'Ethnicity', 'website': 'github.com/istresearch', 'version': '1.0'}, 'results': ['white']}, 'title': {'attribs': {'class': 'custom', 'type': 'title', 'website': 'github.com\\/istresearch', 'version': '1.0', 'target': 'backpage'}, 'results': ['?SLiPP?RY ???????? ? ????? ? ?????? ????????U?iV? BLONDE ?????RATD? - Odessa escorts - backpage.com']}, 'text': {'attribs': {'class': 'custom', 'type': 'text', 'website': 'github.com\\/istresearch', 'version': '1.0', 'target': 'backpage'}, 'results': ["Hey guys I'm Amber I'm a sexy naughty white playmate seeking the company of upscale n@ughty mature gentlemen? <br>\n?INDEPENDENT & PROFESSIONAL SERVICE <br>\n?100% CLEAN&SAFE NO EXCEPTIONS <br>\n?HYGIENE GUARANTEED & EXPECTED <br>\n?Height: 55<br>\n?Weight: 135<br>\n?Breast: 34C (ALL NATURAL!) <br>\n?FOR THE N@UGHTY BOYS ASK FOR AMBER ? <br>\n?NO BLACK MEN OR THUGS? <br>\n<br>\n??832-532-6351?? <br>\n<br>\n??Explicit Content will get you BLOCKED?"]}, 'posttime': {'

# Load similar images info

In [143]:
jsns = [json.loads(x) for x in open('../../data/images_expanded/ht_training_UPDATED_images.jl', 'r')]
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/ht_training_2_images.jl', 'r')])
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/cp1_random.json_images.jl', 'r')])
records_images = format_jsns_to_key_count(jsns)

In [144]:
print(len(records_images))

16572


In [145]:
df_images = pd.DataFrame.from_items(records_images,orient='index',columns=['images_count'])

In [146]:
#print(df_images.head())
print(len(df_images.index.get_values()))

16572


In [150]:
jsns = [json.loads(x) for x in open('../../data/images_expanded/ht_training_UPDATED_similar_images_count_th100_dist_th0.1.jsonl', 'r')]
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/ht_training_2_similar_images_count_th100_dist_th0.1.jsonl', 'r')])
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/cp1_random.json_similar_images_count_th100_dist_th0.1.jsonl', 'r')])
records_images_similar = format_jsns_to_key_count(jsns)
df_images_similar = pd.DataFrame.from_items(records_images_similar,orient='index',columns=['similar_images_count'])

In [151]:
print(len(df_images_similar.index.get_values()))

16572


In [152]:
jsns = [json.loads(x) for x in open('../../data/images_expanded/ht_training_UPDATED_expanded_ads_from_similar_images_count_th100_dist_th0.1.jsonl', 'r')]
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/ht_training_2_expanded_ads_from_similar_images_count_th100_dist_th0.1.jsonl', 'r')])
jsns.extend([json.loads(x) for x in open('../../data/images_expanded/cp1_random.json_expanded_ads_from_similar_images_count_th100_dist_th0.1.jsonl', 'r')])
records_exp_ads_from_simimages = format_jsns_to_key_count(jsns)
df_exp_ads_from_simimages = pd.DataFrame.from_items(records_exp_ads_from_simimages,orient='index',columns=['exp_ads_from_simimages_count'])

                                                    exp_ads_from_simimages_count
16A2C382C32C9B8D8510597C4FC8483717375AD49CDB3D1...                           183
DD80A66ED5E3A506995497DFF2E25E01F49721AA676665D...                           187
1AD112FF2F4197B344BFF1203B53EF63A400FE0F2BAB0ED...                             2
D513648E2D9A427738C80FE627873D7D187C440E9EC2C72...                             0
2EDCF618255FC6ABD82FC1409DE453578D6E6F20E4B2290...                             6


In [153]:
print(len(df_exp_ads_from_simimages.index.get_values()))

16572


## Try to merge with df

In [154]:
df = pd.read_csv('../../data/merged/data_to_use_by_ad_v3.csv')

In [155]:
print(df.columns)

Index(['class', 'phone', 'dd_id', 'price', 'duration_in_mins', 'flag', 'age',
       'area', 'area_type', 'ethnicity', 'cdr_id', 'price_per_min', 'Disease',
       'Year', 'Cases', 'Rate', 'MSA', 'property', 'rape', 'violent',
       'male_wage_mean', 'male_wage_p05', 'male_wage_p10', 'male_wage_p25',
       'male_wage_p50', 'male_wage_p75', 'male_wage_p90', 'male_wage_p95',
       'male_N', 'male_epop', 'qcew_code', 'msa', 'population', 'unemployment',
       'lt_highschool', 'highschool', 'some_college', 'college_plus',
       'frac_white', 'avg_commute', 'female_wage_mean', 'wage_var.wage',
       'female_wage_p05', 'female_wage_p10', 'female_wage_p25',
       'female_wage_p50', 'female_wage_p75', 'female_wage_p90',
       'female_wage_p95', 'female_N', 'wage_sum.wght', 'female_epop',
       'swnauthemp', 'swnftemp', 'ad_p10_msa', 'ad_p90_msa', 'ad_mean_msa',
       'ad_median_msa', 'ad_count_msa', 'ad_p50_msa'],
      dtype='object')


In [156]:
# Merge column "cdr_id" with index of the other data frames
print(len(df.columns),df.columns)
df_with_img_count = pd.merge(df, df_images, how='left', left_on='cdr_id', right_index=True)
df_with_img_count = pd.merge(df_with_img_count, df_exp_ads_from_simimages, how='left', left_on='cdr_id', right_index=True)
df_with_img_count = pd.merge(df_with_img_count, df_images_similar, how='left', left_on='cdr_id', right_index=True)
print(len(df_with_img_count.columns),df_with_img_count.columns)
print(df.shape)
print(df_with_img_count.shape)
# Missing all the negatives?

60 Index(['class', 'phone', 'dd_id', 'price', 'duration_in_mins', 'flag', 'age',
       'area', 'area_type', 'ethnicity', 'cdr_id', 'price_per_min', 'Disease',
       'Year', 'Cases', 'Rate', 'MSA', 'property', 'rape', 'violent',
       'male_wage_mean', 'male_wage_p05', 'male_wage_p10', 'male_wage_p25',
       'male_wage_p50', 'male_wage_p75', 'male_wage_p90', 'male_wage_p95',
       'male_N', 'male_epop', 'qcew_code', 'msa', 'population', 'unemployment',
       'lt_highschool', 'highschool', 'some_college', 'college_plus',
       'frac_white', 'avg_commute', 'female_wage_mean', 'wage_var.wage',
       'female_wage_p05', 'female_wage_p10', 'female_wage_p25',
       'female_wage_p50', 'female_wage_p75', 'female_wage_p90',
       'female_wage_p95', 'female_N', 'wage_sum.wght', 'female_epop',
       'swnauthemp', 'swnftemp', 'ad_p10_msa', 'ad_p90_msa', 'ad_mean_msa',
       'ad_median_msa', 'ad_count_msa', 'ad_p50_msa'],
      dtype='object')
63 Index(['class', 'phone', 'dd_id', 'price',

In [158]:
#print(df_with_img_count.columns)
print(pd.isnull(df_with_img_count.similar_images_count).sum())
#print(df.cdr_id[0])

83215


In [160]:
print(df_with_img_count.loc[:,['cdr_id','class','images_count', 'similar_images_count','exp_ads_from_simimages_count']].head())

                                              cdr_id     class  images_count  \
0  9A1C8836A1FC78FA27E00749F95B6604632E878C038746...  positive           NaN   
1  83460C6DDEC972921F2AC09E5E24ECC857FDAF29F0767C...  positive           7.0   
2  0196C9051EC97B5736A107DAD33CF35DB9CFA78E49C633...  positive           NaN   
3  2EDCF618255FC6ABD82FC1409DE453578D6E6F20E4B229...  positive           2.0   
4  2EDCF618255FC6ABD82FC1409DE453578D6E6F20E4B229...  positive           2.0   

   similar_images_count  exp_ads_from_simimages_count  
0                   NaN                           NaN  
1                 270.0                         278.0  
2                   NaN                           NaN  
3                  12.0                           6.0  
4                  12.0                           6.0  


In [163]:
df_with_img_count.to_csv('../../data/images_expanded/data_to_use_by_ad_v3_with_exp_imgs.csv', index=False)